<a href="https://colab.research.google.com/github/yesyell/DA_School_1/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EA%B3%BC%ED%95%99%20%EB%B0%A9%EB%B2%95%EB%A1%A0/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# 데이터를 pandas DataFrame으로 불러옴
review_data = pd.read_excel("review_20230725_091330.xlsx")

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
import sqlite3

# 1. SQLite3를 사용하여 데이터베이스 파일 생성
conn = sqlite3.connect("reviews.db")

# 2. 해당 데이터베이스에 테이블을 생성하고 데이터를 입력
review_data.to_sql('product_reviews', conn, if_exists='replace', index=False)

# 3. 데이터를 조회하여 확인
query = "SELECT * FROM product_reviews LIMIT 5"
result = pd.read_sql(query, conn)
result.head(2)

,상품번호,상품명,리뷰구분,구매자평점,포토/영상,리뷰상세내용,리뷰도움수,등록자,리뷰등록일,최종수정일,...,답글등록일시,베스트리뷰,베스트리뷰선정일시,이벤트번호,혜택지급,혜택지급일시,신고여부,신고일시,상품주문번호,풀필먼트사
0,8194608691,[대용량/원두커피] 싱글오리진 에티오피아 예가체프 첼베사 / 500g / 커피템플,일반,5,https://phinf.pstatic.net/checkout.phinf/20230...,항상 만족합니다. 맛있어요,None,hp****,2023.07.24. 21:51:55,None,...,None,N,None,None,None,None,N,None,2023071770600621,None
1,8378279956,[드립백커피] 드립백커피 샘플러 / 10gx4ea / 커피템플,일반,5,None,선물하기딱좋을 패키지라 구매햇으요^^^,None,arum****,2023.07.22. 20:19:16,None,...,None,N,None,None,None,None,N,None,2023072017835031,None


## a. 텍스트 데이터에서의 GROUP BY

In [11]:
query = """
    select 상품명, count(리뷰상세내용) as 리뷰개수
    from product_reviews
    group by 상품명
    order by count(리뷰상세내용) desc
    limit 5
"""
result = pd.read_sql(query, conn)
result

,상품명,리뷰개수
0,[대용량 / 원두커피] 쥬시 블렌딩 / 1kg / 커피템플,73
1,[원두커피] 쥬시 블렌딩 / 200g / 커피템플,62
2,[리미티드에디션] 게스트바리스타 / 콰트로 콜롬비아 100g / 스페셜티커피 / 모...,47
3,[원두커피]클래식 블렌딩 / 200g / 커피템플,43
4,[드립백커피] 쥬시 블렌딩 / 10gx10ea / 드립백 세트 / 커피템플,32


In [8]:
# 각 상품별 리뷰 개수 확인
grouped_reviews = review_data.groupby('상품명').size().reset_index(name='리뷰개수')
grouped_reviews.head()

,상품명,리뷰개수
0,[ 8/26 11:00 오픈 ] 게스트바리스타 / 노스텔지아 225g / 스페셜티커...,1
1,[18차] 게스트바리스타 / 노스텔지아 225g / 스페셜티커피 / 카페어니언 / ...,9
2,"[2+1이벤트] 커피템플 대표 드립백커피 4종 쥬시, 클래식, 에티오피아, 디카프 ...",14
3,"[2+1이벤트] 커피템플 대표 캡슐커피 4종 쥬시, 클래식, 에티오피아, 디카프 /...",6
4,[9/30 10시오픈예정] 게스트바리스타 / 댄싱베이비 100g / 스페셜티커피 /...,5


## b. 텍스트 데이터에 대한 COUNT(), SUM(), AVG() 활용

## c. LIKE를 활용한 패턴 검색

In [22]:
query = """
    SELECT count(리뷰상세내용), AVG(구매자평점)
    FROM product_reviews
	WHERE 상품명 LIKE '%쥬시%' and 리뷰상세내용 LIKE '%맛%';
"""
result = pd.read_sql(query, conn)
result

,count(리뷰상세내용),AVG(구매자평점)
0,132,4.94697


In [15]:
# '커피'라는 단어가 들어간 리뷰의 개수 확인
coffee_reviews_count = review_data[review_data['리뷰상세내용'].str.contains('커피', na=False)].shape[0]
coffee_reviews_count

200

In [23]:
# 평균 평점 계산
average_rating = review_data['구매자평점'].mean()
average_rating

4.921161825726141

## d. REGEXP를 활용한 복잡한 패턴 검색

In [27]:
import re

# 1. 정규 표현식 패턴 일치 함수 정의
def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

# 2. 함수를 SQLite 연결에 REGEXP 연산자로 연결
conn.create_function("REGEXP", 2, regexp)

# 이제 REGEXP 연산자를 사용하여 쿼리를 수행할 수 있습니다.
query = """
    SELECT 리뷰상세내용
    FROM product_reviews
    WHERE 리뷰상세내용 REGEXP '커피'
"""
result_regexp = pd.read_sql(query, conn)
result_regexp.head()

,리뷰상세내용
0,박스 뜯자마자 커피 향이 너무 좋더라고요.\n맛도 정말 좋아요! 믿고 먹는 커피템플!
1,커피전문가 추천글보고 구입했는데\n정말 맛있네요!!\n진짜 카페에서 내려먹는 맛이예...
2,클래식 커피를 내려먹어 봤어요.\n쓰거나 신맛없이 부드러운 맛이고 딱 먹기 좋아요.
3,매일마시는커피 넘맛있어용
4,믿고 먹는 커피템플 원두 ㅠㅠ


In [31]:
# 커피 맛별 구매자 평점 집계
query = """
    SELECT
        CASE
            WHEN 리뷰상세내용 LIKE '%쥬시%' THEN '쥬시'
            WHEN 리뷰상세내용 LIKE '%아메리카노%' THEN '아메리카노'
            WHEN 리뷰상세내용 LIKE '%에스프레소%' THEN '에스프레소'
            WHEN 리뷰상세내용 LIKE '%드립백%' THEN '드립백'
            WHEN 리뷰상세내용 LIKE '%캡슐%' THEN '캡슐'
            ELSE '기타'
        END AS 커피맛,
        AVG(구매자평점) AS 평균평점
    FROM product_reviews
    GROUP BY 1
    order by 2 desc;
"""
result = pd.read_sql(query, conn)
result

,커피맛,평균평점
0,아메리카노,5.000000
1,기타,4.929766
2,드립백,4.914286
3,쥬시,4.909091
4,에스프레소,4.909091
5,캡슐,4.774194


In [35]:
# 커피 맛별 일별 평점 집계
query = """
    SELECT SUBSTR(리뷰등록일, 1, 10) AS 날짜,
        CASE
            WHEN 리뷰상세내용 LIKE '%쥬시%' THEN '쥬시'
            WHEN 리뷰상세내용 LIKE '%아메리카노%' THEN '아메리카노'
            WHEN 리뷰상세내용 LIKE '%에스프레소%' THEN '에스프레소'
            WHEN 리뷰상세내용 LIKE '%드립백%' THEN '드립백'
            WHEN 리뷰상세내용 LIKE '%캡슐%' THEN '캡슐'
            ELSE '기타'
        END AS 커피맛,
        AVG(구매자평점) AS 평균평점
    FROM product_reviews
    GROUP BY 1, 2;
"""
result = pd.read_sql(query, conn)
result

,날짜,커피맛,평균평점
0,2022.07.27,기타,5.0
1,2022.07.28,기타,5.0
2,2022.07.29,기타,5.0
3,2022.07.29,쥬시,3.0
4,2022.07.30,기타,5.0
...,...,...,...
367,2023.07.20,쥬시,5.0
368,2023.07.21,기타,5.0
369,2023.07.21,쥬시,5.0
370,2023.07.22,기타,5.0


In [39]:
# 리뷰 구분별 시간별 평점 집계
query = """
    SELECT SUBSTR(리뷰등록일, 16, 2) AS 날짜,
        리뷰구분,
        AVG(구매자평점) AS 평균평점
    FROM product_reviews
    GROUP BY 1, 2;
"""
result = pd.read_sql(query, conn)
result

,날짜,리뷰구분,평균평점
0,00,일반,5.000000
1,01,일반,5.000000
2,01,한달사용,5.000000
3,02,일반,5.000000
4,03,일반,4.666667
...,...,...,...
100,57,일반,5.000000
101,58,일반,5.000000
102,58,한달사용,5.000000
103,59,일반,5.000000


In [42]:
# 리뷰에 '드립백'이라는 맛이 들어간 상품 중 평점이 가장 높은 상품 찾기
query = """
    SELECT 상품명, MAX(구매자평점) AS 최고평점
    FROM product_reviews
    WHERE 리뷰상세내용 LIKE '%드립백%';
"""
result = pd.read_sql(query, conn)
result

,상품명,최고평점
0,[드립백커피] 쥬시 블렌딩 / 10gx10ea / 드립백 세트 / 커피템플,5


In [43]:
# '아메리카노'라는 맛을 포함한 리뷰 중 가장 오래된 리뷰 찾기
query = """
    SELECT 리뷰상세내용, SUBSTR(리뷰등록일, 1, 10) as 일자
    FROM product_reviews
    WHERE 리뷰상세내용 LIKE '%아메리카노%'
    ORDER BY 2
    LIMIT 1;
"""
result = pd.read_sql(query, conn)
result

,리뷰상세내용,일자
0,너무 맛있습니다.\n베르크 게바 현장에서 마셔보고 너무 맛있어서 바로 온라인 구입했...,2022.10.10


https://paullabworkspace.notion.site/52d7e9d2081041ffa40de8d9b7902757